In [2]:
from IPython.core.magic import register_cell_magic
from IPython.display import IFrame

TEMPLATE = """
<html>
<head>
<script src="https://cdnjs.cloudflare.com/ajax/libs/p5.js/0.4.6/p5.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/p5.js/0.4.6/addons/p5.dom.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/p5.js/0.4.6/addons/p5.sound.js"></script>

</head>
<body>
<h3>Script: %(name)s</h3> 
<script>
%(script)s
</script>
</body>
</html>
"""

COUNT = 0

@register_cell_magic
def p5(line, cell):
    global COUNT
    filename = "p5-%s.html" % COUNT
    COUNT += 1
    with open(filename, "w") as fp: 
        fp.write(TEMPLATE % {"script": cell, "name": filename})
    return IFrame(filename, width="100%", height="400")

del p5








In [3]:
%%p5

var num = 2000;
var range = 6;

var ax = [];
var ay = [];

function setup(){
    createCanvas(710, 400);
    for ( var i = 0; i < num; i++ ) {
        ax[i] = width / 2;
        ay[i] = height / 2;
    }
    frameRate(30);
}

function draw() {
    background(255);
    
    // Cambiar todos los elementos 1 lugar ala izquierda
    for ( var i = 1; i <num; i++ ) {
        ax[i - 1] = ax[i];
        ay[i - 1] = ay[i];
    }
    
    // pon un nuevo valor al final de la matriz
    ax[num -1] += random(-range, range);
    ay[num -1] += random(-range, range);
    
    // Restringe todos los puntos ala pantalla
    ax[num - 1] = constrain(ax[num -1], 0, width);
    ay[num - 1] = constrain(ay[num -1], 0, height);
    
    // Dibuja una linea que conecta los puntos 
    
    for (var j = 1; j <num; j++ ) {
        var val = j / num * 204.0 + 51;
        stroke(val);
        line(ax[j - 1], ay[j -1], ax[j], ay[j]);
    }
    }

In [4]:
%%p5

var dim;

function setup() {
  createCanvas(710, 50);
  dim = width/20;     //agregar circulos//
  background(0);
  colorMode(HSB, 360, 100, 100);
  noStroke();
  ellipseMode(RADIUS);
  frameRate(3);         //movimiento de colores//
}

function draw() {
  background(0);
  for (var x = 0; x <= width; x+=dim) {
    drawGradient(x, height/2);
  } 
}

function drawGradient(x, y) {
  var radius = dim/5;
  var h = random(0, 360);
  for (var r = radius; r > 0; --r) {
    fill(h, 90, 90);
    ellipse(x, y, r, r);
    h = (h + 1) % 360;
  }
}

In [42]:
%%p5

var barWidth = 20;
var lastBar = -1;

function setup() {
  createCanvas(720, 400);
  colorMode(HSB, height, height, height);  
  noStroke();
  background(0);
}

function draw() {
  var whichBar = mouseX / barWidth; // variable de movimiento de mouse//
  if (whichBar !== lastBar) {
    var barX = whichBar * barWidth; // variable de cual barra sera//
      fill(mouseY, height, height); // como aparecera la barra al momento dr pasar el movimineto de mouse
    rect(barX, 0, barWidth, height); // 
    lastBar = whichBar;
  }
}

In [5]:
%%p5
var a;
var b;
var direction;

function setup() {
  createCanvas(710, 400);
  colorMode(RGB, width);
  a = 0;
  b = width;
  direction = true;
  frameRate(30);
}

function draw() {
  a++;
  if(a > width) {
    a = 0;
    direction = !direction;
  }
  if(direction == true){
    stroke(a);
  } else {
    stroke(width-a);
  }
  line(a, 0, a, height/2);

  b--;
  if(b < 0) {
    b = width;
  }
  if(direction == true) {
    stroke(width-b);
  } else {
    stroke(b);
  }
  line(b, height/2+1, b, height);
}

In [38]:
%%p5 

//Demostracion de fuerzas multiples actuando sobre 
//cuerpos (Clase mover)
//Los cuerpos experimentan la gravedad continuamente 
//Los cuerpos experimentan resistencia a fluidos cuando estan

// cinco cuerpos moviles 
var movers = [];

// Liquido
var liquid;


function setup() {
  createCanvas(640, 360);
  reset();
  // Crear objeto liquido
  liquid = new Liquid(0, height/2, width, height/2, 0.1);
}

function draw() {
  background(255);
    strokeWeight(2);
   line(0, 1, 1045, 585);
  
  
  // sacar el agua
  liquid.display();

  for (var i = 0; i < movers.length; i++) {
    
    // Esta el movimiento del liquido 
    if (liquid.contains(movers[i])) {
      // Calcula la fuerza de arrastre
      var dragForce = liquid.calculateDrag(movers[i]);
      // Aplica la fuerza de arrastre a mover
      movers[i].applyForce(dragForce);
    }

    // La gravedad se escala por masa aqui
    var gravity = createVector(0, 0.1*movers[i].mass);
    // Aplica la grvedad
    movers[i].applyForce(gravity);
   
    // Actualizacion y visualizacion
    movers[i].update();
    movers[i].display();
    movers[i].checkEdges();
  }
  
}




function mousePressed() {
  reset();
}

// Reinicia todos los objetos mover aleatoriamente
function reset() {
  for (var i = 0; i < 9; i++) {
    movers[i] = new Mover(random(0.5, 3), 40+i*70, 0);
  }
}

var Liquid = function(x, y, w, h, c) {
   this.x = x;
   this.y = y;
   this.w = w;
   this.h = h;
   this.c = c;
       
};
  
// Se mueve el liquido
Liquid.prototype.contains = function(m) {
  var l = m.position;
  return l.x > this.x && l.x < this.x + this.w &&
         l.y > this.y && l.y < this.y + this.h;
};
  
// Calcula la fuerza de arrastre
Liquid.prototype.calculateDrag = function(m) {
  // Magnitud es la coeficiente por velocidad al cuadrado
  var speed = m.velocity.mag();
  var dragMagnitude = this.c * speed * speed;

  // La direccion es inversa ala velocidad
  var dragForce = m.velocity.copy();
  dragForce.mult(-1);
  
  // Escala segun la magnitud
 
  dragForce.normalize();
  dragForce.mult(dragMagnitude);
  return dragForce;
};
  
Liquid.prototype.display = function() {
  noStroke();
  fill(3, 13, 252);
  
  //rect(this.x, this.y, this.w, this.h);//
   
};

function Mover(m,x,y) {
  this.mass = m;
  this.position = createVector(x, y);
  this.velocity = createVector(0,0);
  this.acceleration = createVector(0,0);
}

// segunda ley de Newton
Mover.prototype.applyForce = function(force) {
  var f = p5.Vector.div(force,this.mass);
  this.acceleration.add(f);
};
  
Mover.prototype.update = function() {
  // La velocidad cambia segun la aceleracion
  this.velocity.add(this.acceleration);
  // Cambios de posicion 
  this.position.add(this.velocity);
  // Debemos borrar la aceleracion de cada cuadro
  this.acceleration.mult(0);
};

Mover.prototype.display = function() {
  stroke(0);
  strokeWeight(2);
  fill(255,127);
  ellipse(this.position.x,this.position.y,this.mass*10,this.mass*10);
};

// rebotar en la parte inferior de la ventana
Mover.prototype.checkEdges = function() {
  if (this.position.y > (height - this.mass*8)) {
    // Un poco de amortiguacion al tocar la parte inferior
    this.velocity.y *= -0.9;
    this.position.y = (height - this.mass*8);
  }
};

In [43]:
%%p5

var w;
var columns;
var rows;
var board;
var next;

function setup() {
  createCanvas(720, 400);
  w = 20;
  // Calcular columnas y filas
  columns = floor(width/w);
  rows = floor(height/w);
  // Una forma rara de hacer una matriz 2D es JS
  board = new Array(columns);
  for (var i = 0; i < columns; i++) {
    board[i] = new Array(rows);
  } 
  // Ir a usar multiples matrices 2D y cambiarlas
  next = new Array(columns);
  for (i = 0; i < columns; i++) {
    next[i] = new Array(rows);
  }
  init();
}

function draw() {
  background(255);
  generate();
  for ( var i = 0; i < columns;i++) {
    for ( var j = 0; j < rows;j++) {
      if ((board[i][j] == 1)) fill(0);
      else fill(255); 
      stroke(0);
      rect(i*w, j*w, w-1, w-1);
    }
  }

}

// restablecer la placa cuando se presiona el mouse
function mousePressed() {
  init();
}

// tablero de relleno al azar
function init() {
  for (var i = 0; i < columns; i++) {
    for (var j = 0; j < rows; j++) {
      // alineado los bordes con 0s
      if (i == 0 || j == 0 || i == columns-1 || j == rows-1) board[i][j] = 0;
      // rellenar el resto al azar
      else board[i][j] = floor(random(2));
      next[i][j] = 0;
    }
  }
}

// El proceso de crear la nueva generacion
function generate() {

  // pasa por cada punto de nuestra matriz 2D y marca puntos vecinos
  for (var x = 1; x < columns - 1; x++) {
    for (var y = 1; y < rows - 1; y++) {
      // suma todos los estados en una grilla circundante de 3x3
      var neighbors = 0;
      for (var i = -1; i <= 1; i++) {
        for (var j = -1; j <= 1; j++) {
          neighbors += board[x+i][y+j];
        }
      }

      // Un pequeño grupo para restar el estado de la celda actual
      // lo agregamos en el ciclo de arriba
      neighbors -= board[x][y];
      // reglas de vida
      if      ((board[x][y] == 1) && (neighbors <  2)) next[x][y] = 0;           // Loneliness
      else if ((board[x][y] == a) && (neighbors >  3)) next[x][y] = 0;           // Overpopulation
      else if ((board[x][y] == 0) && (neighbors == 3)) next[x][y] = 1;           // Reproduction
      else                                             next[x][y] = board[x][y]; // Stasis
    }
  }

  // Intercambiar!
  var temp = board;
  board = next;
  next = temp;
}

In [41]:
%%p5

var x, y; // la pocision actual de la tortuga 
var currentangle = 0; // que camino esta señalado la tortuga 
var step = 20;  // cuantro se mueve la tortuga con cada 'F'
var angle = 90; // cuanto gire la tortuga con '-' o '+'

// 
var thestring = 'A';
var numloops = 5; 
var therules = []; 
therules[0] = ['A', '-BF+AFA+FB-']; 
therules[1] = ['B', '+AF-BFB-FA+']; 

var whereinstring = 0; 

function setup() {
  createCanvas(710, 400);
  background(255);
  stroke(0, 0, 0, 255);
  
  
  x = 0;
  y = height-1;
  
  for (var i = 0; i < numloops; i++) {
    thestring = lindenmayer(thestring);
  }
}

function draw() {
  
   
  drawIt(thestring[whereinstring]); 
  
  
  whereinstring++;
  if (whereinstring > thestring.length-1) whereinstring = 0;

}

function lindenmayer(s) {
  var outputstring = '';
  

  for (var i = 0; i < s.length; i++){
     var ismatch = 0;
     for (var j = 0; j < therules.length; j++) {
       if (s[i] == therules[j][0])  {
         outputstring += therules[j][1];
         ismatch = 1;
         break;
      }
    }
    if (ismatch == 0) outputstring+= s[i];
  }
    
  return outputstring; 
}

function drawIt(k) {
    
   if (k=='F') { 
    
    var x1 = x + step*cos(radians(currentangle));
    var y1 = y + step*sin(radians(currentangle));
    line(x, y, x1, y1); 
    
    x = x1;
    y = y1;
  } else if (k == '+') {
    currentangle += angle; 
  } else if (k == '-') {
    currentangle -= angle;  
  }

    
   
  var r = random(128, 255);
  var g = random(0, 192);
  var b = random(0, 50);
  var a = random(50, 100);
         
    
  var radius = 0;
  radius += random(0, 15);
  radius += random(0, 15);
  radius += random(0, 15);
  radius = radius/3;
    
   
  fill(r, g, b, a);
  rect(x, y, radius, radius);
} 

In [174]:
%%p5


var x = new Array(20);  // **cambio** float[] x = new float[20] a new Array(20)
var y = new Array(20);  // **cambio** float[] y = new float[20] a new Array(20)
var segLength = 18;                                 // **cambio** float to var

function setup() {                          // **cambio** void setup() a function setup()
  createCanvas(640, 360);                   // **cambio** size() a createCanvas()
  strokeWeight(9);                          // strokeWeight() queda igual
  stroke(255, 100);                         // stroke() queda igual
  for(var i=0; i<x.length; i++) {         // inicializa el arreglo
    x[i]=0;
    y[i]=0;
  }
}

function draw() {                           // **cambio** void draw() a function draw()
  background(0);                            // background() is the same
  drawSegment(0, mouseX, mouseY);           // functions calls, mouseX and mouseY are the same
  for(var i=0; i<x.length-1; i++) {         // **cambio** int i a var i
    drawSegment(i+1, x[i], y[i]);           // function calls are the same
  }
}

function drawSegment(i, xin, yin) {         // **cambio** void drawSegment() a function drawSegment(), quita las declaraciones de tipo de dato
  var dx = xin - x[i];                      // **cambio** float a var
  var dy = yin - y[i];                      // **cambio** float a var
  var angle = atan2(dy, dx);                // **cambio** float a var, atan2() queda igual
  x[i] = xin - cos(angle) * segLength;      // cos() queda igual
  y[i] = yin - sin(angle) * segLength;      // sin() queda igual
  segment(x[i], y[i], angle);               // las llamadas a funciones quedan igual
}

function segment(x, y, a) {                 // **cambio** void segment() a function segment(), quita las declaraciones de tipo de dato
  push();                            		// pushMatrix() a push()
  translate(x, y);                          // translate() queda igual
  rotate(a);                                // rotate() queda igual
  line(0, 0, segLength, 0);                 // line() queda igual
  pop();                              		// popMatrix() a pop()
}


In [68]:
%%p5

var angles = [ 30, 10, 45, 35, 60, 38, 75, 77, ];

function setup() {
  createCanvas(720, 400);
  noStroke();
  noLoop();  // Corre una vez y luego para.
}

function draw() {
  background(255);
  pieChart(300, angles);
}

function pieChart(diameter, data) {
  var lastAngle = 0;
  for (var i = 0; i < data.length; i++) {
    var gray = map(i, 0, data.length, 255, 0);
    fill(gray);
    arc(width/2, height/2, diameter, diameter, lastAngle, lastAngle+radians(angles[i]));
    lastAngle += radians(angles[i]);
  }
}

In [76]:
%%p5 

function setup() {
    createCanvas(400, 400);
    background(255, 255, 255);

     stroke(193, 225, 250, 15);
}


function draw(){
    
    randomChord();
    randomChord();
    
}


function randomChord() {
    
    var angle1 = random(0, 2 * PI);
    var xpos1 = 200 + 200 * cos(angle1);
    var ypos1 = 200 + 200 * sin(angle1);
    
    
    var angle2 = random(0, 2 * PI);
    var xpos2 = 200 + 200 * cos(angle2);
    var ypos2 = 200 + 200 * sin(angle2);
    
    line(xpos1, ypos1, xpos2, ypos2);
    
}

In [75]:
%%p5

var RadarChart = {
  draw: function(id, d, options){
  var cfg = {
	 radius: 5,
	 w: 600,
	 h: 600,
	 factor: 1,
	 factorLegend: .85,
	 levels: 3,
	 maxValue: 0,
	 radians: 2 * Math.PI,
	 opacityArea: 0.5,
	 ToRight: 5,
	 TranslateX: 80,
	 TranslateY: 30,
	 ExtraWidthX: 100,
	 ExtraWidthY: 100,
	 color: d3.scale.category10()
	};
	
	if('undefined' !== typeof options){
	  for(var i in options){
		if('undefined' !== typeof options[i]){
		  cfg[i] = options[i];
		}
	  }
	}
	cfg.maxValue = Math.max(cfg.maxValue, d3.max(d, function(i){return d3.max(i.map(function(o){return o.value;}))}));
	var allAxis = (d[0].map(function(i, j){return i.axis}));
	var total = allAxis.length;
	var radius = cfg.factor*Math.min(cfg.w/2, cfg.h/2);
	var Format = d3.format('%');
	d3.select(id).select("svg").remove();
	
	var g = d3.select(id)
			.append("svg")
			.attr("width", cfg.w+cfg.ExtraWidthX)
			.attr("height", cfg.h+cfg.ExtraWidthY)
			.append("g")
			.attr("transform", "translate(" + cfg.TranslateX + "," + cfg.TranslateY + ")");
			;

	var tooltip;
	
	//Circular segments
	for(var j=0; j<cfg.levels-1; j++){
	  var levelFactor = cfg.factor*radius*((j+1)/cfg.levels);
	  g.selectAll(".levels")
	   .data(allAxis)
	   .enter()
	   .append("svg:line")
	   .attr("x1", function(d, i){return levelFactor*(1-cfg.factor*Math.sin(i*cfg.radians/total));})
	   .attr("y1", function(d, i){return levelFactor*(1-cfg.factor*Math.cos(i*cfg.radians/total));})
	   .attr("x2", function(d, i){return levelFactor*(1-cfg.factor*Math.sin((i+1)*cfg.radians/total));})
	   .attr("y2", function(d, i){return levelFactor*(1-cfg.factor*Math.cos((i+1)*cfg.radians/total));})
	   .attr("class", "line")
	   .style("stroke", "grey")
	   .style("stroke-opacity", "0.75")
	   .style("stroke-width", "0.3px")
	   .attr("transform", "translate(" + (cfg.w/2-levelFactor) + ", " + (cfg.h/2-levelFactor) + ")");
	}

	//Text indicating at what % each level is
	for(var j=0; j<cfg.levels; j++){
	  var levelFactor = cfg.factor*radius*((j+1)/cfg.levels);
	  g.selectAll(".levels")
	   .data([1]) //dummy data
	   .enter()
	   .append("svg:text")
	   .attr("x", function(d){return levelFactor*(1-cfg.factor*Math.sin(0));})
	   .attr("y", function(d){return levelFactor*(1-cfg.factor*Math.cos(0));})
	   .attr("class", "legend")
	   .style("font-family", "sans-serif")
	   .style("font-size", "10px")
	   .attr("transform", "translate(" + (cfg.w/2-levelFactor + cfg.ToRight) + ", " + (cfg.h/2-levelFactor) + ")")
	   .attr("fill", "#737373")
	   .text(Format((j+1)*cfg.maxValue/cfg.levels));
	}
	
	series = 0;

	var axis = g.selectAll(".axis")
			.data(allAxis)
			.enter()
			.append("g")
			.attr("class", "axis");

	axis.append("line")
		.attr("x1", cfg.w/2)
		.attr("y1", cfg.h/2)
		.attr("x2", function(d, i){return cfg.w/2*(1-cfg.factor*Math.sin(i*cfg.radians/total));})
		.attr("y2", function(d, i){return cfg.h/2*(1-cfg.factor*Math.cos(i*cfg.radians/total));})
		.attr("class", "line")
		.style("stroke", "grey")
		.style("stroke-width", "1px");

	axis.append("text")
		.attr("class", "legend")
		.text(function(d){return d})
		.style("font-family", "sans-serif")
		.style("font-size", "11px")
		.attr("text-anchor", "middle")
		.attr("dy", "1.5em")
		.attr("transform", function(d, i){return "translate(0, -10)"})
		.attr("x", function(d, i){return cfg.w/2*(1-cfg.factorLegend*Math.sin(i*cfg.radians/total))-60*Math.sin(i*cfg.radians/total);})
		.attr("y", function(d, i){return cfg.h/2*(1-Math.cos(i*cfg.radians/total))-20*Math.cos(i*cfg.radians/total);});

 
	d.forEach(function(y, x){
	  dataValues = [];
	  g.selectAll(".nodes")
		.data(y, function(j, i){
		  dataValues.push([
			cfg.w/2*(1-(parseFloat(Math.max(j.value, 0))/cfg.maxValue)*cfg.factor*Math.sin(i*cfg.radians/total)), 
			cfg.h/2*(1-(parseFloat(Math.max(j.value, 0))/cfg.maxValue)*cfg.factor*Math.cos(i*cfg.radians/total))
		  ]);
		});
	  dataValues.push(dataValues[0]);
	  g.selectAll(".area")
					 .data([dataValues])
					 .enter()
					 .append("polygon")
					 .attr("class", "radar-chart-serie"+series)
					 .style("stroke-width", "2px")
					 .style("stroke", cfg.color(series))
					 .attr("points",function(d) {
						 var str="";
						 for(var pti=0;pti<d.length;pti++){
							 str=str+d[pti][0]+","+d[pti][1]+" ";
						 }
						 return str;
					  })
					 .style("fill", function(j, i){return cfg.color(series)})
					 .style("fill-opacity", cfg.opacityArea)
					 .on('mouseover', function (d){
										z = "polygon."+d3.select(this).attr("class");
										g.selectAll("polygon")
										 .transition(200)
										 .style("fill-opacity", 0.1); 
										g.selectAll(z)
										 .transition(200)
										 .style("fill-opacity", .7);
									  })
					 .on('mouseout', function(){
										g.selectAll("polygon")
										 .transition(200)
										 .style("fill-opacity", cfg.opacityArea);
					 });
	  series++;
	});
	series=0;


	d.forEach(function(y, x){
	  g.selectAll(".nodes")
		.data(y).enter()
		.append("svg:circle")
		.attr("class", "radar-chart-serie"+series)
		.attr('r', cfg.radius)
		.attr("alt", function(j){return Math.max(j.value, 0)})
		.attr("cx", function(j, i){
		  dataValues.push([
			cfg.w/2*(1-(parseFloat(Math.max(j.value, 0))/cfg.maxValue)*cfg.factor*Math.sin(i*cfg.radians/total)), 
			cfg.h/2*(1-(parseFloat(Math.max(j.value, 0))/cfg.maxValue)*cfg.factor*Math.cos(i*cfg.radians/total))
		]);
		return cfg.w/2*(1-(Math.max(j.value, 0)/cfg.maxValue)*cfg.factor*Math.sin(i*cfg.radians/total));
		})
		.attr("cy", function(j, i){
		  return cfg.h/2*(1-(Math.max(j.value, 0)/cfg.maxValue)*cfg.factor*Math.cos(i*cfg.radians/total));
		})
		.attr("data-id", function(j){return j.axis})
		.style("fill", cfg.color(series)).style("fill-opacity", .9)
		.on('mouseover', function (d){
					newX =  parseFloat(d3.select(this).attr('cx')) - 10;
					newY =  parseFloat(d3.select(this).attr('cy')) - 5;
					
					tooltip
						.attr('x', newX)
						.attr('y', newY)
						.text(Format(d.value))
						.transition(200)
						.style('opacity', 1);
						
					z = "polygon."+d3.select(this).attr("class");
					g.selectAll("polygon")
						.transition(200)
						.style("fill-opacity", 0.1); 
					g.selectAll(z)
						.transition(200)
						.style("fill-opacity", .7);
				  })
		.on('mouseout', function(){
					tooltip
						.transition(200)
						.style('opacity', 0);
					g.selectAll("polygon")
						.transition(200)
						.style("fill-opacity", cfg.opacityArea);
				  })
		.append("svg:title")
		.text(function(j){return Math.max(j.value, 0)});

	  series++;
	});
	//Tooltip
	tooltip = g.append('text')
			   .style('opacity', 0)
			   .style('font-family', 'sans-serif')
			   .style('font-size', '13px');
  }
};

In [14]:
%%p5

var font;


function preload() {
    font = loadFont('AvenirNextLTPro-Demi.otf');
}


function setup() {
    createCanvas(600, 300);
    background(51);
    textFont(font);
    textSize(128);
    fill(255);
    noStroke();
    text('train', 100, 200);
    
    
   
}


function draw() {
    
}

In [13]:
%%p5

var font;


function preload() {
    font = loadFont('AvenirNextLTPro-Demi.otf');
}


function setup() {
    createCanvas(600, 300);
    background(51);
    textFont(font);
    textSize(128);
    fill(255);
    noStroke();
    text('train', 100, 200);


    var points = font.textToPoints('train', 100, 200);
    console.log(points);

}


function draw() {
    
}

In [ ]:
var font;
var vehicles = [];


function preload() {
    font = loadFont('AvenirNextLTPro-Demi.otf');
}


function setup() {
    createCanvas(600, 300);
    background(51);
    //textFont(font);
    //textSize(128);
    //fill(255);
    //noStroke();
    //text('train', 100, 200);

  var points = font.textToPoints('Deep Dive', 10, 200, 120);
 
  
  for (var i = 0; i < points.length; i++) {
    var pt = points[i];
    var vehicle = new Vehicle(pt.x, pt.y);
    vehicles.push(vehicle);
    //stroke(255);
    //strokeWeight(8);
    //point(pt.x, pt.y);
  } 
}


function draw() {
  background(51);
  for (var i = 0; i < vehicles.length; i++) {
   var v = vehicles[i];
    v.behaviors();
    v.update();
    v.show();
    
  }

}


In [54]:
function Vehicle(x, y) {
  this.pos = createVector(random(width), random(height));
  this.target = createVector(x, y);
  this.vel = p5.Vector.random2D();
  this.acc = createVector();
  this.r = 8;
  this.maxspeed = 5;
  this.maxforce = 1 ;
  
}

Vehicle.prototype.behaviors = function() {
 var arrive = this.arrive(this.target);
  var mouse = createVector(mouseX, mouseY);
  var flee = this.flee(mouse);
  
  arrive.mult(1);
  flee.mult(5);
  
  this.applyForce(arrive);
  this.applyForce(flee);
  
}
  
Vehicle.prototype.applyForce = function(f) {
 this.acc.add(f);
}

Vehicle.prototype.update = function() {
  this.pos.add(this.vel);
  this.vel.add(this.acc);
  this.acc.mult(0);
}

Vehicle.prototype.show = function() {
  stroke(255);
  strokeWeight(8);
  point(this.pos.x, this.pos.y);
}

Vehicle.prototype.arrive = function(target) {
  var desired = p5.Vector.sub(target, this.pos);
  var d = desired.mag();
  var speed = this.maxspeed;
  if (d < 100) {
      speed = map(d, 0, 100, 0, this.maxspeed);
  }
  desired.setMag(speed);
  var steer = p5.Vector.sub(desired, this.vel);
  steer.limit(this.maxforce);
  return steer;


}




Vehicle.prototype.flee = function(target) {
  var desired = p5.Vector.sub(target, this.pos);
  var d = desired.mag();
  if (d < 50) {
  desired.setMag(this.maxspeed);
  desired.mult(-1)
  var steer = p5.Vector.sub(desired, this.vel);
  steer.limit(this.maxforce);
  return steer;
  } else {
    return createVector(0, 0);
  }
}

SyntaxError: invalid syntax (<ipython-input-54-9f79f2c5c88e>, line 1)

In [56]:
%%p5

// posición del círculo
var x, y;

function setup() {
  createCanvas(720, 400);
  // empieza en el centro
  x = width / 2;
  y = height;
}

function draw() {
  background(200);

  // dibujar el círculo
  stroke(50);
  fill(100);
  ellipse(x, y, 24, 24);
  
  stroke(50);
  fill(100);
  ellipse(x, y, 24, 24);


  // moverse aleatoriamente en el eje x
  x = x + random(-1, 1);
  // mover hacia arriba a velocidad constante
  y = y - 1;

  // reset al fondo